In [1]:
from pathlib import Path


# out_file_name = "raw_questions"
# out_file_name = "raw_questions_2"
out_file_name = "raw_questions_wiki_sci_2"


out_dir = Path(f"../data/data_dumps/{out_file_name}")

# if the directory already exist, raise an error
# out_dir.mkdir(exist_ok=True, parents=True)

In [2]:
import json


formatted_answers = []


answer_files = sorted(list(out_dir.glob("*.txt")))

for p in answer_files:
    try:
        parsed_file = json.loads(p.read_text())
        formatted_answers.append(parsed_file)
    except json.JSONDecodeError as e:
        print(f"cant decode: {p}, {e}")

print(f"parsed {len(formatted_answers)} / {len(answer_files)} files")

cant decode: ../data/data_dumps/raw_questions_wiki_sci_2/cluster-1-round-3-676581-3.txt, Expecting property name enclosed in double quotes: line 89 column 7 (char 4124)
cant decode: ../data/data_dumps/raw_questions_wiki_sci_2/cluster-107-round-4-508103-3.txt, Expecting ',' delimiter: line 10 column 7 (char 403)
cant decode: ../data/data_dumps/raw_questions_wiki_sci_2/cluster-114-round-3-252560-3.txt, Expecting ',' delimiter: line 16 column 23 (char 803)
cant decode: ../data/data_dumps/raw_questions_wiki_sci_2/cluster-124-round-4-902028-3.txt, Expecting property name enclosed in double quotes: line 88 column 6 (char 2836)
cant decode: ../data/data_dumps/raw_questions_wiki_sci_2/cluster-150-round-4-916667-3.txt, Unterminated string starting at: line 9 column 482 (char 3078)
cant decode: ../data/data_dumps/raw_questions_wiki_sci_2/cluster-160-round-7-199534-3.txt, Expecting ',' delimiter: line 19 column 7 (char 1208)
cant decode: ../data/data_dumps/raw_questions_wiki_sci_2/cluster-161-rou

In [3]:
import pandas as pd
import json
from pathlib import Path

# Get a sorted list of all ".txt" files in the directory
answer_files = sorted(list(out_dir.glob("*.txt")))

failed_reads = 0
data_list = []

# Loop through each file
for p in answer_files:
    try:
        # Extract cluster number and round from the filename
        cluster_num = int(p.stem.split('-')[1])
        round_num = int(p.stem.split('-')[3])

        # Load file content with json.loads
        parsed_file = json.loads(p.read_text())

        for q in parsed_file["questions"]:
            data_list.append({
                "id": None,  # You'll need to decide how you want to generate the IDs
                "prompt": q["question"],
                "A": q["A"],
                "B": q["B"],
                "C": q["C"],
                "D": q["D"],
                "E": q["E"],
                "answer": q["answer"],
                "cluster number": cluster_num,
                "round": round_num
            })

    except Exception as e:
        # print(f"Failed reading file: {p.name}. Error: {e}")
        failed_reads += 1

print(f"Failed reads: {failed_reads}")

# Convert data_list into a dataframe
df = pd.DataFrame(data_list)

# Assign unique IDs to each row
df["id"] = range(1, len(df) + 1)



Failed reads: 325


In [4]:
# convert prompt	A	B	C	D	E	answer	 columns to string
df["prompt"] = df["prompt"].astype(str)
df["A"] = df["A"].astype(str)
df["B"] = df["B"].astype(str)
df["C"] = df["C"].astype(str)
df["D"] = df["D"].astype(str)
df["E"] = df["E"].astype(str)
df["answer"] = df["answer"].astype(str)

In [5]:
df

,id,prompt,A,B,C,D,E,answer,cluster number,round
0,1,What is the significance of HD 87883 b?,It is the closest extrasolar planet to Earth.,It is the first planet discovered in the const...,It has the longest orbital period of any known...,It was the first planet detected by the radial...,It has a highly eccentric orbit around its star.,E,0,0
1,2,Which of the following statements about HD 878...,It orbits a K-type main sequence star.,It takes less than a year to complete one orbi...,It was discovered using the transit method.,It has an average distance from its star of 1....,It was detected in 2020.,A,0,0
2,3,What is the definition of a long-period planet?,A planet that takes more than 10 years to comp...,A planet that takes less than a year to comple...,A planet that has an average distance from its...,A planet that has a highly eccentric orbit.,A planet that orbits a K-type main sequence star.,A,0,0
3,4,What is the difference between the closest and...,0.20 AU,1.82 AU,3.82 AU,4.82 AU,6.82 AU,D,0,0
4,5,What is the reason behind the eccentric orbit ...,Gravitational interactions with other planets ...,The tidal forces exerted by the star on the pl...,The presence of a second star in the system.,The planet's initial velocity at the time of f...,Radiation pressure from the star.,A,0,0
...,...,...,...,...,...,...,...,...,...,...
95322,95323,What is the definition of quasiperiodic light ...,Light curves that are irregular and change sli...,Light curves that have magnitudes less than 0.5,Light curves that exhibit periodic variations ...,Light curves that have active chromospheres,Light curves that are classified as K or M stars,A,999,7
95323,95324,What is the purpose of studying BY Draconis va...,To identify nearby K and M stars,To discover new categories of variable stars,To observe flares resulting in additional vari...,To investigate the similarities between BY Dra...,To gain insight into the variability mechanism...,E,999,7
95324,95325,What is the reason behind the similarity betwe...,They both have irregular light curves,They both exhibit variations in luminosity due...,They both have periodic light curves,They both have magnitudes less than 0.5,They both have active chromospheres,B,999,7
95325,95326,What is the term used to describe the variabil...,Quasiperiodic,Chromospheric,Flares,Spectra,Main sequence,A,999,7


In [6]:
# is there any none in the df?
df.isna().sum()

id                0
prompt            0
A                 0
B                 0
C                 0
D                 0
E                 0
answer            0
cluster number    0
round             0
dtype: int64

In [7]:
df

,id,prompt,A,B,C,D,E,answer,cluster number,round
0,1,What is the significance of HD 87883 b?,It is the closest extrasolar planet to Earth.,It is the first planet discovered in the const...,It has the longest orbital period of any known...,It was the first planet detected by the radial...,It has a highly eccentric orbit around its star.,E,0,0
1,2,Which of the following statements about HD 878...,It orbits a K-type main sequence star.,It takes less than a year to complete one orbi...,It was discovered using the transit method.,It has an average distance from its star of 1....,It was detected in 2020.,A,0,0
2,3,What is the definition of a long-period planet?,A planet that takes more than 10 years to comp...,A planet that takes less than a year to comple...,A planet that has an average distance from its...,A planet that has a highly eccentric orbit.,A planet that orbits a K-type main sequence star.,A,0,0
3,4,What is the difference between the closest and...,0.20 AU,1.82 AU,3.82 AU,4.82 AU,6.82 AU,D,0,0
4,5,What is the reason behind the eccentric orbit ...,Gravitational interactions with other planets ...,The tidal forces exerted by the star on the pl...,The presence of a second star in the system.,The planet's initial velocity at the time of f...,Radiation pressure from the star.,A,0,0
...,...,...,...,...,...,...,...,...,...,...
95322,95323,What is the definition of quasiperiodic light ...,Light curves that are irregular and change sli...,Light curves that have magnitudes less than 0.5,Light curves that exhibit periodic variations ...,Light curves that have active chromospheres,Light curves that are classified as K or M stars,A,999,7
95323,95324,What is the purpose of studying BY Draconis va...,To identify nearby K and M stars,To discover new categories of variable stars,To observe flares resulting in additional vari...,To investigate the similarities between BY Dra...,To gain insight into the variability mechanism...,E,999,7
95324,95325,What is the reason behind the similarity betwe...,They both have irregular light curves,They both exhibit variations in luminosity due...,They both have periodic light curves,They both have magnitudes less than 0.5,They both have active chromospheres,B,999,7
95325,95326,What is the term used to describe the variabil...,Quasiperiodic,Chromospheric,Flares,Spectra,Main sequence,A,999,7


In [8]:
out_path = f"../data/data_dumps/more_questions/more_questions_{out_file_name}.csv"
print(out_path)
df.to_csv(out_path)

../data/data_dumps/more_questions/more_questions_raw_questions_wiki_sci_2.csv
